## 不含模型参数的自定义层

In [1]:
import torch
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self,**kwargs):
        super(CenteredLayer,self).__init__(**kwargs)
    def forward(self,x):
        return x-x.mean()

In [2]:
layer=CenteredLayer()
layer(torch.tensor([1,2,3,4,5],dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

In [3]:
net=nn.Sequential(nn.Linear(8,128),CenteredLayer())

In [4]:
y=net(torch.rand(4,8))
print(y.shape)
y.mean().item()

torch.Size([4, 128])


0.0

## 含模型参数的自定义层

In [5]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense,self).__init__()
        self.params=nn.ParameterList([nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4,1)))
    def forward(self,x):
        for i in range(len(self.params)):
            x=torch.mm(x,self.params[i])
        return x
net=MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


In [6]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense,self).__init__()
        self.params=nn.ParameterDict({
            'linear1':nn.Parameter(torch.randn(4,4)),
            'linear2':nn.Parameter(torch.randn(4,1))
        })
        self.params.update({'linear3':nn.Parameter(torch.randn(4,2))})
    def forward(self,x,choice='linear1'):
        return torch.mm(x,self.params[choice])
net=MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [7]:
x=torch.ones(1,4)
print(net(x,'linear1'))
print(net(x,'linear2'))
print(net(x,'linear3'))

tensor([[-2.9082,  0.9834,  0.0164, -0.2384]], grad_fn=<MmBackward>)
tensor([[1.6447]], grad_fn=<MmBackward>)
tensor([[ 1.8851, -0.9588]], grad_fn=<MmBackward>)


In [8]:
net=nn.Sequential(
    MyDictDense(),
    MyListDense(),
)
print(net)
print(net(x))

Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): MyListDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)
tensor([[22.1480]], grad_fn=<MmBackward>)
